In [1]:
import hnswlib
import numpy as np 
import struct
import heapq

In [ ]:
def distributed_search(query_vec, kmeans, index_list, k, ef, all_vectors):
    """
    query_vec: a numpy array of a single d-dimensional vector
    kmeans: the kmeans object
    index_list: a list of loaded python object of hnsw index
    """
    
    partition_id = k_means.predict(query_vec)
    search_path = []
    all_local_results = []
    
    while True:
        current_index = index_list[partition_id]
        search_path.append(partition_id)
        
        # TODO: Here, need also return remote partition ID
        results, local_results, remote_results, search_remote, remote_partition_id = \ 
            current_index.searchKnnPlusRemoteCache(query_vec, k, ef, all_vectors, debug=False)
        all_local_results += local_results
        if not search_remote:
            break
        else:
            if remote_partition_id in search_path:
                break
            else:
                partition_id = remote_partition_id
                
    # merge all results
    results_heap = []
    for dist, ID in all_local_results:
        heapq.heappush(results_heap, (-dist, ID))
    while len(results_heap) > k:
        heapq.heappop(results_heap)

    results = []
    while len(results_heap) > 0:
        dist, ID = results_heap[0]
        results.append((-dist, ID))
        heapq.heappop(results_heap)
    results.reverse()
    
    # this is in descending order for distance
    return results, search_path